In [2]:
from typing import NoReturn
import logging

import numpy as np
from numpy.typing import ArrayLike

import scipy as sp

In [3]:
def convm(x: ArrayLike, p: int) -> np.ndarray:
    """Construct the convolution matrix of the signal x with p number of parameters.

    (N + p - 1) by p non-symmetric Toeplitz matrix
    """
    _x = np.array(x, dtype=complex).ravel()
    if p < 1:
        raise ValueError(f"{p=} must be greater or equal to 1.")

    N = len(_x) + 2 * p - 2
    # the signal centered over its support
    # needed for the signal information-preserving frequency spectrum
    xcol = (_x.copy()).reshape(-1, 1)
    logging.warning(f'\n{xcol=}')
    xpad = np.concatenate((np.zeros((p - 1, 1)), xcol, np.zeros((p - 1, 1))))
    logging.warning(f'{xpad=}\n')
    X = np.empty([len(_x) + p - 1, p], dtype=complex)
    for i in range(p):
        X[:, i] = xpad[p - i - 1:N - i, 0]
    return X

In [4]:
def covar(x: ArrayLike, p: int) -> np.ndarray:
    """Covariance Matrix.

    p x p hermitian toeplitz matrix of sample covariances
    """
    _x = np.array(x, dtype=complex)
    m = len(_x)
    # remove the mean
    #_x = _x - np.mean(_x)
    conv = convm(x, p + 1)
    logging.warning(f'\n{conv.shape=}')
    logging.warning(f'\n{conv=}')
    R = conv.conjugate().transpose() @ conv.copy() / (m - 1)
    return R

In [5]:
from pprint import pprint
p = 2
x = np.array([6, 1.92705 + 4.58522j, -3.42705 + 3.49541j], dtype=complex)
#x = np.array([6, 1 + 4j, -2 + 3j], dtype=complex)
R = np.sqrt(covar(x, p))
pprint(f'{R.shape=}')
pprint(f'{R=}')

xcol=array([[ 6.     +0.j     ],
       [ 1.92705+4.58522j],
       [-3.42705+3.49541j]])
       [ 0.     +0.j     ],
       [ 6.     +0.j     ],
       [ 1.92705+4.58522j],
       [-3.42705+3.49541j],
       [ 0.     +0.j     ],
       [ 0.     +0.j     ]])

conv.shape=(5, 3)
conv=array([[ 6.     +0.j     ,  0.     +0.j     ,  0.     +0.j     ],
       [ 1.92705+4.58522j,  6.     +0.j     ,  0.     +0.j     ],
       [-3.42705+3.49541j,  1.92705+4.58522j,  6.     +0.j     ],
       [ 0.     +0.j     , -3.42705+3.49541j,  1.92705+4.58522j],
       [ 0.     +0.j     ,  0.     +0.j     , -3.42705+3.49541j]])


'R.shape=(3, 3)'
('R=array([[6.50770032+0.j        , 4.33516862-2.88114099j,\n'
 '        1.48396786-3.53317288j],\n'
 '       [4.33516862+2.88114099j, 6.50770032+0.j        ,\n'
 '        4.33516862-2.88114099j],\n'
 '       [1.48396786+3.53317288j, 4.33516862+2.88114099j,\n'
 '        6.50770032+0.j        ]])')


In [6]:
d, v = np.linalg.eigh(R)
logging.warning(f'{d=}')
logging.warning(f'{v=}')

        -5.59440906e-01+0.00000000e+00j],
       [ 6.58941580e-01+4.37886114e-01j, -1.43317534e-05+2.15652010e-05j,
        -5.09363821e-01-3.38526577e-01j],
       [-1.67514374e-01-3.98703910e-01j,  2.73852671e-01+6.51923856e-01j,
        -2.16643073e-01-5.15790564e-01j]])


In [7]:
_x = np.array(x, ndmin=2)
_xt = _x.transpose()
print(_xt.shape)
print(_xt)
np.dot(_xt, _x.conjugate())

(3, 1)
[[ 6.     +0.j     ]
 [ 1.92705+4.58522j]
 [-3.42705+3.49541j]]


array([[ 36.         +0.j        ,  11.5623    -27.51132j   ,
        -20.5623    -20.97246j   ],
       [ 11.5623    +27.51132j   ,  24.73776415 +0.j        ,
          9.42312714-22.44960804j],
       [-20.5623    +20.97246j   ,   9.42312714+22.44960804j,
         23.96256277 +0.j        ]])

In [8]:
_x = convm(x, 3)
print(_x)

xcol=array([[ 6.     +0.j     ],
       [ 1.92705+4.58522j],
       [-3.42705+3.49541j]])
       [ 0.     +0.j     ],
       [ 6.     +0.j     ],
       [ 1.92705+4.58522j],
       [-3.42705+3.49541j],
       [ 0.     +0.j     ],
       [ 0.     +0.j     ]])



[[ 6.     +0.j       0.     +0.j       0.     +0.j     ]
 [ 1.92705+4.58522j  6.     +0.j       0.     +0.j     ]
 [-3.42705+3.49541j  1.92705+4.58522j  6.     +0.j     ]
 [ 0.     +0.j      -3.42705+3.49541j  1.92705+4.58522j]
 [ 0.     +0.j       0.     +0.j      -3.42705+3.49541j]]


In [9]:
print(_x.transpose().conjugate())
myx = _x.transpose().conjugate()
mx = _x[:3,:3] + myx[:,:3]
mx

[[ 6.     -0.j       1.92705-4.58522j -3.42705-3.49541j  0.     -0.j
   0.     -0.j     ]
 [ 0.     -0.j       6.     -0.j       1.92705-4.58522j -3.42705-3.49541j
   0.     -0.j     ]
 [ 0.     -0.j       0.     -0.j       6.     -0.j       1.92705-4.58522j
  -3.42705-3.49541j]]


array([[12.     +0.j     ,  1.92705-4.58522j, -3.42705-3.49541j],
       [ 1.92705+4.58522j, 12.     +0.j     ,  1.92705-4.58522j],
       [-3.42705+3.49541j,  1.92705+4.58522j, 12.     +0.j     ]])

In [10]:
tt = np.array([6, 1.92705 + 4.58522j, -3.42705 + 3.49541j], dtype=complex)
Rx = sp.linalg.toeplitz(tt)
Rx

array([[ 6.     +0.j     ,  1.92705-4.58522j, -3.42705-3.49541j],
       [ 1.92705+4.58522j,  6.     +0.j     ,  1.92705-4.58522j],
       [-3.42705+3.49541j,  1.92705+4.58522j,  6.     +0.j     ]])

In [11]:
d, v = np.linalg.eigh(Rx)
print(d)
print(Rx)
print(v)

[ 1.00000161  1.10485869 15.8951397 ]
[[ 6.     +0.j       1.92705-4.58522j -3.42705-3.49541j]
 [ 1.92705+4.58522j  6.     +0.j       1.92705-4.58522j]
 [-3.42705+3.49541j  1.92705+4.58522j  6.     +0.j     ]]
[[ 0.40974303+0.j         -0.7070716 +0.j         -0.57633359+0.j        ]
 [-0.33145704-0.74455189j -0.0091694 +0.00392879j -0.22439919-0.53415747j]
 [-0.2741981 +0.30447455j -0.48772554+0.51193168j  0.40342281-0.41159499j]]


In [12]:

ddiag = np.diag(d)
logging.warning(f'{ddiag=}')
index = np.argmin(d)
print(index)
vmin = v[:, index]
sigma = d[index]
logging.warning(f'{v=}')
logging.warning(f'{vmin=}')
logging.warning(f'{sigma=}')

       [ 0.        ,  1.10485869,  0.        ],
       [ 0.        ,  0.        , 15.8951397 ]])
        -0.57633359+0.j        ],
       [-0.33145704-0.74455189j, -0.0091694 +0.00392879j,
        -0.22439919-0.53415747j],
       [-0.2741981 +0.30447455j, -0.48772554+0.51193168j,
         0.40342281-0.41159499j]])
       -0.2741981 +0.30447455j])


0


In [13]:
rts = np.roots(vmin)
print(f'{rts=}')
print(f'{np.angle(rts)=}')

rts=array([0.49995334+0.86605234j, 0.30898547+0.95106676j])
np.angle(rts)=array([1.04725142, 1.25667021])


In [52]:
sigspace = v[:,index+2]
vsig = v[:,index+1:].copy()
#sigspace = sigspace / np.abs(sigspace)
print(sigspace)

[-0.57633359+0.j         -0.22439919-0.53415747j  0.40342281-0.41159499j]


In [53]:
phases = np.angle(rts)
print(np.exp(np.concatenate(([0], phases))))
sigexp = np.concatenate(([1], np.exp([1j * phases[0]]),
					   np.exp([2j * phases[0]])), dtype=complex)
sigexp = np.exp(np.concatenate(([0], [1j * phases[0]], [2j * phases[0]])))
print(sigexp)
res =  sigspace @ sigexp.conjugate()
print(res)
print(np.abs(res))
print(np.abs(res)**2)

[1.         2.84980743 3.51370211]
[ 1.        +0.j          0.49995334+0.86605234j -0.50009331+0.86597153j]
(-1.709309626520612-0.21622913400032218j)
1.7229319306654465
2.9684944377065627


In [70]:
siggie = np.exp(1j * np.arange(3).reshape(-1, 1) * phases)
dtft = vsig.transpose() @ siggie.conjugate()
dtft = np.abs(dtft)**2

In [71]:
dsig = d[index+1:] - d[index]
dsig 

array([ 0.10485709, 14.8951381 ])

In [72]:
np.linalg.solve(dtft)

TypeError: solve() missing 2 required positional arguments: 'a' and 'b'